In [1]:
import numpy as np
import pandas as pd

from os.path import join, exists, isfile #, isdir, dirname, exists
#from os import mkdir

import pickle

from src.preprocess.stim_times import *

In [2]:
# Project path
raw_path = '/home/climbach/approach-retreat/data/raw'
interim_path = '/home/climbach/approach-retreat/data/interim'
processed_path = '/home/climbach/approach-retreat/data/processed'

yoked = pd.read_excel(join(raw_path,'CON_yoked_table.xlsx'))
yoked = yoked.query('use == 1')

run_vars = ['run%s' % i for i in range(6)]
control = yoked[['control'] + run_vars]
uncontrol = yoked[['uncontrol'] + run_vars]

In [3]:
# Define path to approach timings
appr_file_path = join(raw_path,'{subj}/splitted_regs_fancy/{subj}_closeAllApprMR.txt')
# Define path to retreat timings
retr_file_path = join(raw_path,'{subj}/splitted_regs_fancy/{subj}_closeAllRetrMR.txt')
# Path to experimental proximity values
proximity_path = join(raw_path,'{subj}/regs_fancy/{subj}_all_regs.txt')

# Define path to the shock beta map of every subject
cortical_path = join(interim_path,'{subj}/{subj}_cortical_meanTS.1D')
subcortical_path = join(interim_path,'{subj}/{subj}_subcortical_meanTS.1D')

In [4]:
if not exists(join(processed_path,'00a-ROI316_withShiftedSegments.pkl')):
    print('File does not exist')

In [5]:
if exists(join(processed_path,'00a-ROI316_withShiftedSegments.pkl')):
    pass
else:
    SEGMENTS= dict(); mean_TS = []
    for _,row in yoked.iterrows():
        for kind in ['uncontrol']:
            run_list = np.arange(6)[row.loc['run0':'run5'].astype(bool)]
            nruns = len(run_list)
            subj=row[kind]
            print(subj)

            # Loading timeseries
            cortical_ts = np.loadtxt(cortical_path.format(subj=subj))
            subcortical_ts = np.loadtxt(subcortical_path.format(subj=subj))
            ts = np.concatenate((cortical_ts,subcortical_ts),axis=1)
            # Splitting timeseries into runs and z-scoring within subject, run, ROI
            runs = [(X - X.mean(axis=0))/X.std(axis=0) for X in np.split(ts,nruns,axis=0)]

            mean_TS.append(np.vstack(runs).mean(axis=0))

            # Reading the proximity regressor and splitting it into runs
            prox = np.split(pd.read_csv(proximity_path.format(subj=subj),sep='\t',index_col=0).iloc[:,0].values, nruns,axis=0)

            # Reading the approach and retreat onset-duration files
            approach_file = open(appr_file_path.format(subj=subj),'r').readlines()
            retreat_file = open(retr_file_path.format(subj=subj),'r').readlines()
            all_seg = []; all_labels = []
            for run in range(len(runs)):
                ap_on, ap_du = getSelectedStims(approach_file[run])
                peak = ap_on+ap_du
                re_on, re_du = getSelectedStims(retreat_file[run])
                ap_idx = getApprRetrCorrespondingIdx(peak,re_on)
                _,peakTRs = return_closest_TR(peak[ap_idx])

                print('\tRun: ')
                print('\t\tmean = 0: ',np.allclose(np.zeros_like(runs[run].mean(axis=0)),runs[run].mean(axis=0)))
                print('\t\tstd = 1: ',np.allclose(np.ones_like(runs[run].std(axis=0)),runs[run].std(axis=0)))
                # Reponse for the peak approach will be shifted by 3.75 seconds (4TRs), so lets add 4 to peakTRs

                labels = np.stack([prox[run][indx-6:indx+8] for indx in peakTRs],axis=1)
                all_labels.append(labels)

                peakTRs += 3
                peakTRs = peakTRs[peakTRs >= 12] # remove very early peakTRs so we don't run out of index when shifting back in time
                seg = []
                for dt in range(0,6):
                    shifted_peakTRs = peakTRs - dt
                    seg.append(np.stack([runs[run][indx-6:indx+8,:] for indx in shifted_peakTRs],axis=2))
                seg = np.dstack([el for el in seg]) # change axis=3 to keep the shifted segements in a separate dim 
                #seg = np.stack([el for el in seg],axis=3)
                all_seg.append(seg)
            SEGMENTS[subj] = np.dstack(all_seg)
            SEGMENTS[subj] = {'data':np.dstack(all_seg),'target':np.hstack(all_labels)}
            
    with open(join(processed_path,'00a-ROI316_withShiftedSegments.pkl'),'wb') as file:
        pickle.dump(SEGMENTS,file)